# Generate tables in density and temperatures for the extended SCvH EOS

The original SCvH EOS table was extended to lower temperatures (and pressures?). However, these tables use $(P, T)$ as variables which is not useful for applications in hydro codes. 

We therefore have to invert $\rho \left( P, T \right)$ to $ P\left( \rho, T \right)$ by solving
$$ \rho \left( P, T=const \right) - \rho_i = 0$$
for a grid of $\rho_i$ using a root finder.


### Import modules

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import interpolate as interp
#from scipy import optimize
import numpy as np

In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'

In [ ]:
mpl.rcParams['figure.dpi'] = 100

Convert units from cgs to SI:

In [ ]:
press_unit_si = 0.1        # erg/cm^3 in Pascal
spec_energy_unit_si = 1e4  # erg/g in J/kg

In [ ]:
data = np.loadtxt("hydrogen_scvh_extended.data") 

logT_table   = data[:, 0] 
logP_table   = data[:, 1]
frac_H2      = data[:, 2]
frac_H       = data[:, 3]
logrho_table = data[:, 4]
logu_table   = data[:, 5]
logs_table   = data[:, 6]

# All the SCvH EOS tables are tabulated along isotherms
logT_table_axis = np.unique(logT_table)
nT = np.size(logT_table_axis)

print("Number of isotherms: nT = {:}".format(nT))

# The number of grid points in P are different for each isotherm
logP_table_axis = list()
logrho_isotherm = list()
logu_isotherm = list()
logs_isotherm = list()

for logT in logT_table_axis:
    logP_table_axis.append(logP_table[np.where(logT_table == logT)])
    logrho_isotherm.append(logrho_table[np.where(logT_table == logT)])
    logu_isotherm.append(logu_table[np.where(logT_table == logT)])
    logs_isotherm.append(logs_table[np.where(logT_table == logT)])

logT_min   = np.min(logT_table_axis)
logT_max   = np.max(logT_table_axis)

print("logT_min = {:}".format(logT_min))
print("logT_max = {:}".format(logT_max))
print()

# Mark where the original SCvH EOS table was extended below T=100K
index = np.min(np.where(logT_table_axis >= 2.0))
print("The EOS table was extended until index={:} logT={:}".format(index, logT_table_axis[index]))


In [ ]:
# Generate interpolation functions for each isotherm

logrho_of_P_int = list()
logu_of_P_int = list()
logs_of_P_int = list()

for i in range(0, nT):
    logrho_of_P_int.append(interp.InterpolatedUnivariateSpline(logP_table_axis[i], logrho_isotherm[i], k=1))
    logu_of_P_int.append(interp.InterpolatedUnivariateSpline(logP_table_axis[i], logu_isotherm[i], k=1))
    logs_of_P_int.append(interp.InterpolatedUnivariateSpline(logP_table_axis[i], logs_isotherm[i], k=1))


First plot the table and check if the result looks sensible.

In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 5
nSkipRho = 10

In [ ]:
# Plot log(rho), log(u) and log(s)
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# rho(P, T=const)
for i in range(0, nT, nSkipT):
    ax[0][0].plot(logP_table_axis[i], logrho_isotherm[i], '-')
    
ax[0][0].plot(logP_table_axis[0], logrho_isotherm[0], '--', color='black', label=r"T$_{min}$")

    
ax[0][0].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(rho) [g cm^${-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

"""
# P(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[0][1].plot(logT_table, logP[:,i], '-')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")
"""

# u(rho, T=const)
for i in range(0, nT, nSkipT):
    ax[1][0].plot(logrho_table, logu[i,:], '-')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[1][1].plot(logT_table, logu[:,i], '-')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, nT, nSkipT):
    ax[2][0].plot(logrho_table, logs[i,:], '-')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[2][1].plot(logT_table, logs[:,i], '-')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")
